<a href="https://colab.research.google.com/github/CatherineFC/NLP/blob/main/Clasificador_Glosa_Bert_transformers_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  !pip install datasets transformers

In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
!pip install transformers[torch]>=4.6.0

In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, pipeline
import numpy as np


In [ ]:
glosas_categorias = load_dataset('cfigueroa/glosas_etiquetadas')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for cfigueroa/glosas_etiquetadas contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cfigueroa/glosas_etiquetadas.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
glosas_categorias

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 23021
    })
})

In [ ]:
print("Claves del diccionario:", list(glosas_categorias.keys()))

Claves del diccionario: ['train', 'test']


In [ ]:
metric = load_metric('accuracy')

<ipython-input-8-343a7ea47872>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
tokenized_glosas_categorias = glosas_categorias.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/23021 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=40)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_glosas_categorias

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 23021
    })
})

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    max_steps=500,
    eval_strategy="steps"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer = tokenizer,
    train_dataset=tokenized_glosas_categorias["train"],
    eval_dataset=tokenized_glosas_categorias["test"],
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:

# Entrenar el modelo
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,1.351200,3.650421,0.235220


TrainOutput(global_step=500, training_loss=1.3512010498046876, metrics={'train_runtime': 90.7146, 'train_samples_per_second': 44.094, 'train_steps_per_second': 5.512, 'total_flos': 36260026204032.0, 'train_loss': 1.3512010498046876, 'epoch': 0.1})

In [ ]:
classifier = pipeline("text-classification", model = model, tokenizer=tokenizer, device=0)

In [ ]:
classifier(["pan marraqueta","leche descremada 1 lt","Galletas SodaMckay","vestido lino fino", "CARTERA DE CUERO","pañales babysec talla pequeño"])

[{'label': 'LABEL_30', 'score': 0.7355026006698608},
 {'label': 'LABEL_21', 'score': 0.90549236536026},
 {'label': 'LABEL_1', 'score': 0.8756494522094727},
 {'label': 'LABEL_36', 'score': 0.7176593542098999},
 {'label': 'LABEL_14', 'score': 0.4175349473953247},
 {'label': 'LABEL_38', 'score': 0.29462090134620667}]

In [ ]:
classifier(["300grs queso parmesano", "audif izq negro","plateada 4kg", "cualquier cosa"])

[{'label': 'LABEL_1', 'score': 0.9665046334266663},
 {'label': 'LABEL_36', 'score': 0.6088539958000183},
 {'label': 'LABEL_19', 'score': 0.4248083531856537},
 {'label': 'LABEL_36', 'score': 0.4013458490371704}]

In [ ]:
classifier(["trineo", "vinilo","sillón terraza", "cabernet souvignon","libro bolaños","Whisky", "ksd idid"])

[{'label': 'LABEL_36', 'score': 0.4745365083217621},
 {'label': 'LABEL_10', 'score': 0.29857203364372253},
 {'label': 'LABEL_10', 'score': 0.20063824951648712},
 {'label': 'LABEL_37', 'score': 0.9076113700866699},
 {'label': 'LABEL_10', 'score': 0.1748926192522049},
 {'label': 'LABEL_23', 'score': 0.7424300909042358},
 {'label': 'LABEL_20', 'score': 0.245741605758667}]

In [ ]:
# Guarda el modelo y el tokenizer
model.save_pretrained("./my_model",safe_serialization=False)
tokenizer.save_pretrained("./my_model",safe_serialization=False)

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json',
 './my_model/tokenizer.json')

In [ ]:
import safetensors

In [ ]:
# Guardar el modelo y el tokenizer en formato safetensors
model.save_pretrained("./my_model", safe=True)
tokenizer.save_pretrained("./my_model")

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json',
 './my_model/tokenizer.json')

In [ ]:
import shutil

shutil.make_archive("my_model", 'zip', "./my_model")

'/content/my_model.zip'

In [ ]:
from google.colab import files

files.download("my_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')